In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

# import matplotlib.pyplot as plt

import librosa
import librosa.display
import IPython.display as ipd

# import torch
# import lightning
# import transformers

# lightning.__version__, torch.__version__, transformers.__version__

## Right Now Path -- RH standard?

In [2]:
!pwd

/home/heiscold/prac


## Basic Path 

In [3]:
lang = 'german'
print(f"LANGUAGE: {lang}")
speech_data_base_path= f'/data/speech-data/mls/mls_{lang}_opus/' ## train, test, dev 
train_data_path = speech_data_base_path + 'train/audio/' ## there're folders whose names are number.
speech_data_base_path, train_data_path

LANGUAGE: german


('/data/speech-data/mls/mls_german_opus/',
 '/data/speech-data/mls/mls_german_opus/train/audio/')

## transcription info

In [4]:
train_transcripts_path = speech_data_base_path + 'train/transcripts.txt'

ft= open(train_transcripts_path, 'r')
line = ft.readline()
# print(line)
line

'10148_10119_000000\tmein freund sie dürfen ihn nicht einen augenblick verlassen sie sagten er habe die absicht zu erkennen gegeben mich diesen abend zu sprechen ja madame es handelt sich ohne zweifel darum ihre lage in england zu sichern\n'

### Little Sample for txt file info

In [5]:
file_id, sentence = line.split('\tmein')
sentence = sentence.strip().replace('\n', '')
file_id, sentence 

('10148_10119_000000',
 'freund sie dürfen ihn nicht einen augenblick verlassen sie sagten er habe die absicht zu erkennen gegeben mich diesen abend zu sprechen ja madame es handelt sich ohne zweifel darum ihre lage in england zu sichern')

In [6]:
file_id = line.split()[0]
sentence = " ".join(line.split()[1:]).strip()
file_id, sentence 

('10148_10119_000000',
 'mein freund sie dürfen ihn nicht einen augenblick verlassen sie sagten er habe die absicht zu erkennen gegeben mich diesen abend zu sprechen ja madame es handelt sich ohne zweifel darum ihre lage in england zu sichern')

## DataFrame for Sentence

In [7]:
train_transcripts_path = speech_data_base_path + 'train/transcripts.txt'
print(train_transcripts_path)

ft= open(train_transcripts_path, 'r')
id_sentences = []
## takes some time...? 
for i, line in enumerate(ft.readlines()):
    id_sentences.append( (line.split()[0], " ".join(line.split()[1:]).strip()) )
    # if i == 10:
    #     break

print(len(id_sentences))
print(id_sentences[:2])

/data/speech-data/mls/mls_german_opus/train/transcripts.txt
469942
[('10148_10119_000000', 'mein freund sie dürfen ihn nicht einen augenblick verlassen sie sagten er habe die absicht zu erkennen gegeben mich diesen abend zu sprechen ja madame es handelt sich ohne zweifel darum ihre lage in england zu sichern'), ('10148_10119_000001', 'wie heißt dieser dampfer fragte herr fogg carnatic versetzte der lootse sollte er nicht schon gestern abfahren ja mein herr aber es mußte einer seiner kessel reparirt werden und seine abfahrt wurde auf morgen verschoben')]


In [8]:
df2 = pd.DataFrame()

df2['file_id'] = [s[0] for s in id_sentences]
df2['sentence'] = [s[1] for s in id_sentences]
df2['speaker_id']= [tuple(file_id.split("_"))[0] for file_id in df2.file_id.to_list()]
df2['chapter_id']= [tuple(file_id.split("_"))[1] for file_id in df2.file_id.to_list()]
df2['utterance_id']= [str(tuple(file_id.split("_"))[2]) for file_id in df2.file_id.to_list()]

print(df2.shape)
df2.head()

(469942, 5)


,file_id,sentence,speaker_id,chapter_id,utterance_id
0,10148_10119_000000,mein freund sie dürfen ihn nicht einen augenbl...,10148,10119,000000
1,10148_10119_000001,wie heißt dieser dampfer fragte herr fogg carn...,10148,10119,000001
2,10148_10119_000002,war die gelassene antwort,10148,10119,000002
3,10148_10119_000003,noch wirksamer als amphion's leier sollte die ...,10148,10119,000003
4,10148_10119_000004,mitunter zeigte sich eine große schar bison di...,10148,10119,000004


## Rerange

In [9]:
df2.columns

Index(['file_id', 'sentence', 'speaker_id', 'chapter_id', 'utterance_id'], dtype='object')

In [10]:
## Rerange
df2 = df2.sort_values(by=[df2.columns[2],df2.columns[3],df2.columns[4]],ascending=True)
df2 = df2.reset_index(drop = True) # index reset
df2 = df2[['file_id', 'speaker_id', 'chapter_id', 'utterance_id',  'sentence']] # column rerange
print(df2.shape)
df2.head()

(469942, 5)


,file_id,speaker_id,chapter_id,utterance_id,sentence
0,10087_10388_000000,10087,10388,000000,in deutschland protestierten friedrich wilhelm...
1,10087_10388_000001,10087,10388,000001,bald feuerwerk bald florett bald knotenstock u...
2,10087_10388_000002,10087,10388,000002,oder von stefan zweigs kultiviertem kosmopolit...
3,10087_10388_000003,10087,10388,000003,und nun rührt es sich überall in deutschland s...
4,10087_10388_000004,10087,10388,000004,ich weine weil man so allein ist man kann nich...


## About Audio Path

In [11]:
all_train_audio_paths = []
train_audio_path = [(str(p.stem), train_data_path) for p in Path(train_data_path).glob('*/*/*' + ".opus")]
all_train_audio_paths.extend(train_audio_path)
all_train_audio_paths = sorted(all_train_audio_paths)
len(all_train_audio_paths)

469942

## Let's Make PART

In [12]:
print(f"All Train Data: {len(all_train_audio_paths)}")
print(int(len(all_train_audio_paths) * .01))
# train_audio_paths[0]

# 0 ~ 108080
train_audio_paths = all_train_audio_paths[:int(len(all_train_audio_paths) * .01)]
train_audio_paths[0]

All Train Data: 469942
4699


('10087_10388_000000', '/data/speech-data/mls/mls_german_opus/train/audio/')

## Sample: Sentence(=Utterance), Audio

In [13]:
file_id = train_audio_paths[0][0] # '10001_8844_000000'
path= train_audio_paths[0][1] # '/data/speech-data/mls/mls_english_opus/train/audio/'
dir_audio = ".opus"
dir_txt = speech_data_base_path + 'train/transcripts.txt'

## SPlit
speaker_id, chapter_id, utterance_id = file_id.split("_")
print(f'FILE ID: {file_id}, PATH: {path}')
print(f'SPEAKER_ID: {speaker_id}, CHAPTER_ID:{chapter_id}, UTTER_ID:{utterance_id}')

### TEXT
file_text = speaker_id + "_" + chapter_id + dir_txt
file_text = os.path.join(path, speaker_id, chapter_id, file_text)
print(f'FILE TEXT PATH: {file_text}') ### This Exists

## AUDIO
fileid_audio = speaker_id + "_" + chapter_id + "_" + utterance_id
file_audio = fileid_audio + dir_audio
file_audio = os.path.join(path, speaker_id, chapter_id, file_audio)
print(f'File Audio PATH:{file_audio}') ### This Exists

FILE ID: 10087_10388_000000, PATH: /data/speech-data/mls/mls_german_opus/train/audio/
SPEAKER_ID: 10087, CHAPTER_ID:10388, UTTER_ID:000000
FILE TEXT PATH: /data/speech-data/mls/mls_german_opus/train/audio/10087/10388/10087_10388/data/speech-data/mls/mls_german_opus/train/transcripts.txt
File Audio PATH:/data/speech-data/mls/mls_german_opus/train/audio/10087/10388/10087_10388_000000.opus


In [14]:
# https://github.com/ming024/FastSpeech2/blob/master/config/LJSpeech/preprocess.yaml
sample_rate = 22050 

import torchaudio

# audio, sr = librosa.load(file_audio, sr = None)
audio, sr  = torchaudio.load(file_audio,)
## returned sr is 'orig_sr' of sound
print("ORIGINAL SAMPLE RATE: ", sr)
print("TARGET SAMPLE RATE", sample_rate)

## SAMPLE
# audio = librosa.resample(audio, orig_sr= sr, target_sr= sample_rate )
resample = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate,)
audio = resample(audio)
ipd.display(ipd.Audio(audio, rate = sample_rate))

ORIGINAL SAMPLE RATE:  48000
TARGET SAMPLE RATE 22050


## Let's make DataFrame for easy Access
 - caution) Pytorch Dataset may not need this 

In [15]:
## Make Simple CSV
df1 = pd.DataFrame()
df1['file_id'] = [s[0] for s in all_train_audio_paths]
df1['speaker_id']= [tuple(file_id.split("_"))[0] for file_id in df1.file_id.to_list()]
df1['chapter_id']= [tuple(file_id.split("_"))[1] for file_id in df1.file_id.to_list()]
df1['utterance_id']= [str(tuple(file_id.split("_"))[2]) for file_id in df1.file_id.to_list()]

print(df1.shape)
df1.columns

(469942, 4)


Index(['file_id', 'speaker_id', 'chapter_id', 'utterance_id'], dtype='object')

In [16]:
### Auio Column
df1['audio_path'] = df1.file_id.apply(lambda x: os.path.join(path, x.split('_')[0], x.split('_')[1], x.split('_')[0] + "_" + x.split('_')[1] + "_" + x.split('_')[2] +  ".opus"))
df1 = df1.sort_values(by=[df1.columns[1],df1.columns[2],df1.columns[3]],ascending=True)
df1 = df1.reset_index(drop = True) # index reset
print(df1.shape)
df1.head()

(469942, 5)


,file_id,speaker_id,chapter_id,utterance_id,audio_path
0,10087_10388_000000,10087,10388,000000,/data/speech-data/mls/mls_german_opus/train/au...
1,10087_10388_000001,10087,10388,000001,/data/speech-data/mls/mls_german_opus/train/au...
2,10087_10388_000002,10087,10388,000002,/data/speech-data/mls/mls_german_opus/train/au...
3,10087_10388_000003,10087,10388,000003,/data/speech-data/mls/mls_german_opus/train/au...
4,10087_10388_000004,10087,10388,000004,/data/speech-data/mls/mls_german_opus/train/au...


## Check: Audio-Sentence

In [17]:
idx = 0 

audio_path = df1.audio_path.values[idx]
sentence = df2.sentence.values[idx]

print(f"Audio Path: {audio_path}")
print(f"SENTENCE: {sentence}")

audio, sr  = torchaudio.load(audio_path,)
## returned sr is 'orig_sr' of sound
print("ORIGINAL SAMPLE RATE: ", sr)
print("TARGET SAMPLE RATE", sample_rate)

## SAMPLE
# audio = librosa.resample(audio, orig_sr= sr, target_sr= sample_rate )
resample = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate,)
audio = resample(audio)
ipd.display(ipd.Audio(audio, rate = sample_rate))

Audio Path: /data/speech-data/mls/mls_german_opus/train/audio/10087/10388/10087_10388_000000.opus
SENTENCE: in deutschland protestierten friedrich wilhelm foerster der pädagoge und der arzt nicolai in frankreich erhebt seine stimme vom gebrüll der kriegspropaganda umtobt
ORIGINAL SAMPLE RATE:  48000
TARGET SAMPLE RATE 22050


## Concat: df1, df2

In [18]:
df1.shape[0] == df2.shape[0]

True

In [26]:
df = pd.concat([df2, df1.audio_path], axis =1)
df = df.reset_index(drop = True)
print(df.shape)
df.head()

(469942, 6)


,file_id,speaker_id,chapter_id,utterance_id,sentence,audio_path
0,10087_10388_000000,10087,10388,000000,in deutschland protestierten friedrich wilhelm...,/data/speech-data/mls/mls_german_opus/train/au...
1,10087_10388_000001,10087,10388,000001,bald feuerwerk bald florett bald knotenstock u...,/data/speech-data/mls/mls_german_opus/train/au...
2,10087_10388_000002,10087,10388,000002,oder von stefan zweigs kultiviertem kosmopolit...,/data/speech-data/mls/mls_german_opus/train/au...
3,10087_10388_000003,10087,10388,000003,und nun rührt es sich überall in deutschland s...,/data/speech-data/mls/mls_german_opus/train/au...
4,10087_10388_000004,10087,10388,000004,ich weine weil man so allein ist man kann nich...,/data/speech-data/mls/mls_german_opus/train/au...


## One More Check!

In [27]:
idx = 111
audio_path = df.audio_path.values[idx]
sentence = df.sentence.values[idx]

print(f"Audio Path: {audio_path}")
print(f"SENTENCE: {sentence}")

audio, sr  = torchaudio.load(audio_path,)
## returned sr is 'orig_sr' of sound
print("ORIGINAL SAMPLE RATE: ", sr)
print("TARGET SAMPLE RATE", sample_rate)

## SAMPLE
# audio = librosa.resample(audio, orig_sr= sr, target_sr= sample_rate )
resample = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate,)
audio = resample(audio)
ipd.display(ipd.Audio(audio, rate = sample_rate))

Audio Path: /data/speech-data/mls/mls_german_opus/train/audio/10087/8747/10087_8747_000026.opus
SENTENCE: sie erinnere sich daß als sie bei der villa engelhorn vorüberkamen die uhr fünf minuten über sechs zeigte sie habe aber gehört daß diese uhr sehr vorgegangen sei
ORIGINAL SAMPLE RATE:  48000
TARGET SAMPLE RATE 22050


In [28]:
df = df.sort_values(by=[df.columns[1],df.columns[2], df.columns[3]],ascending=True)
print(df.shape)
df.head()

(469942, 6)


,file_id,speaker_id,chapter_id,utterance_id,sentence,audio_path
0,10087_10388_000000,10087,10388,000000,in deutschland protestierten friedrich wilhelm...,/data/speech-data/mls/mls_german_opus/train/au...
1,10087_10388_000001,10087,10388,000001,bald feuerwerk bald florett bald knotenstock u...,/data/speech-data/mls/mls_german_opus/train/au...
2,10087_10388_000002,10087,10388,000002,oder von stefan zweigs kultiviertem kosmopolit...,/data/speech-data/mls/mls_german_opus/train/au...
3,10087_10388_000003,10087,10388,000003,und nun rührt es sich überall in deutschland s...,/data/speech-data/mls/mls_german_opus/train/au...
4,10087_10388_000004,10087,10388,000004,ich weine weil man so allein ist man kann nich...,/data/speech-data/mls/mls_german_opus/train/au...


## Memory and info

In [29]:
## for check RAM... but...	
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469942 entries, 0 to 469941
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   file_id       469942 non-null  object
 1   speaker_id    469942 non-null  object
 2   chapter_id    469942 non-null  object
 3   utterance_id  469942 non-null  object
 4   sentence      469942 non-null  object
 5   audio_path    469942 non-null  object
dtypes: object(6)
memory usage: 295.4 MB


In [30]:
## for check RAM... but...	
df1.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469942 entries, 0 to 469941
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   file_id       469942 non-null  object
 1   speaker_id    469942 non-null  object
 2   chapter_id    469942 non-null  object
 3   utterance_id  469942 non-null  object
 4   audio_path    469942 non-null  object
dtypes: object(5)
memory usage: 177.7 MB


In [31]:
## for check RAM... but...	
df2.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 469942 entries, 0 to 469941
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   file_id       469942 non-null  object
 1   speaker_id    469942 non-null  object
 2   chapter_id    469942 non-null  object
 3   utterance_id  469942 non-null  object
 4   sentence      469942 non-null  object
dtypes: object(5)
memory usage: 229.8 MB


## Save

In [32]:
!pwd

/home/heiscold/prac


In [33]:
### SAVE
df.to_csv("train_mls_german.csv", index = False)
print("Save Completed")

Save Completed


In [34]:
### SAVE
part = df[:200000].reset_index(drop = True)
part.to_csv("train_part_20k_german.csv", index = False)
print("Save Completed")

Save Completed
